In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict


In [ ]:
class BatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int
    sr: float # strike rate
    bpb: float  # balls per boundary
    boundary_percent: float # percentage of balls that are boundaries
    summary: str

In [ ]:
def calculate_sr(state: BatsmanState) -> BatsmanState:
    sr = (state['runs'] / state['balls']) * 100
    state['sr'] = sr
    # return state  # donts end full state only send computed values
    return {'sr' : sr}  # return the computed value only  ( partial update)

def calculate_bpb(state: BatsmanState) -> BatsmanState:
    bpb = state['balls'] / (state['fours'] + state['sixes'])
    state['bpb'] = bpb
    # return state
    return {'bpb': bpb} # partial update

def calculate_boundary_percent(state: BatsmanState) -> BatsmanState:
    boundary_percent = (((state['fours'] * 4) + (state['sixes'] * 6)) / state['runs'] ) * 100
    state['boundary_percent'] = boundary_percent
    # return state
    return {'boundary_percent': boundary_percent} # partial update


def summary(state: BatsmanState) -> BatsmanState:
    summary  = (f"Batsman Summary:\nStrike Rate: {state['sr']}\nBalls per Boundary: {state['bpb']}\nBoundary Percentage: {state['boundary_percent']}")
    state['summary'] = summary
    # return state
    return {'summary': summary} # partial update

In [ ]:
# define graph
graph = StateGraph(BatsmanState)
# add nodes
graph.add_node('calculate_sr',calculate_sr)
graph.add_node('calculate_bpb',calculate_bpb)
graph.add_node('calculate_boundary_percent',calculate_boundary_percent)
graph.add_node('calculate_summary',summary)

# add edges
graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percent')


graph.add_edge('calculate_sr', 'summary')
graph.add_edge('calculate_bpb','summary')
graph.add_edge('calculate_boundary_percent','summary')


graph.add_edge('summary', END)


# compile graph

workflow = graph.compile()
# print(workflow)



In [ ]:
workflow

In [ ]:
# execute graph
initial_state: BatsmanState = {
    'runs': 120,
    'balls': 80,
    'fours': 10,
    'sixes': 5
}

workflow.invoke(initial_state)